<a href="https://colab.research.google.com/github/pranavkarnani/ResearchPaper-NER/blob/master/roberta_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.8 MB/s 
     |████████████████████████████████| 7.6 MB 50.2 MB/s 
     |████████████████████████████████| 163 kB 62.1 MB/s 


In [2]:
! unzip -q '/content/drive/MyDrive/11711/hw2/ANLP Assignment2/tokenized_papers.zip'

In [3]:
! rm -r /content/tokenized_papers/.ipynb_checkpoints
! rm /content/tokenized_papers/289.json

rm: cannot remove '/content/tokenized_papers/.ipynb_checkpoints': No such file or directory


In [4]:
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling, pipeline, Trainer, TrainingArguments
import torch
import pandas as pd
import os
import io
import json
from tqdm import tqdm

In [5]:
config = {
    'epochs': 2,
    'save_steps': 5000,
    'checkpoint_dir': '/content/drive/MyDrive/11711/hw2/ANLP Assignment2/roberta_mlm_checkpoints/'
}

In [6]:
mlm_pipeline = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [8]:
tokenized_papers_path = '/content/tokenized_papers/'
class ACLAnthologyDataset(Dataset):

    def __init__(self):
        self.paper_names = []
        self.X = []
        for _, file in enumerate(tqdm(sorted(os.listdir(tokenized_papers_path), key = lambda x: int(x.strip('.json'))))):
            if _ >= 200:
                break
            self.paper_names.append(file)
            x = ""
            with open(tokenized_papers_path + file, 'r', encoding='utf-8') as f:
                for line in json.loads(f.readlines()[0])['text'].strip(' ').strip('.').split('.'):
                    if len(line) + len(x) <= 800:
                        x += line
                    else:
                        self.X.append(tokenizer(x, truncation = True))
                        x = line

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

In [9]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
dataset = ACLAnthologyDataset()

 69%|██████▉   | 200/289 [18:46<08:21,  5.63s/it]


In [10]:
training_args = TrainingArguments(
    output_dir=config['checkpoint_dir'],
    overwrite_output_dir=True,
    num_train_epochs=config['epochs'],
    per_device_train_batch_size=8,
    save_steps=config['save_steps'],
    save_total_limit=None,
    seed=1,
    disable_tqdm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()

trainer.save_model("./roberta-retrained")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1001417
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 250356


Step,Training Loss
500,2.328300
1000,2.052300
1500,1.948100
2000,1.868000
2500,1.810500
3000,1.770500
